<a href="https://colab.research.google.com/github/Int314/techgym-python/blob/master/techgym_python_04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import random

players = []
table = []
cells = []

# プレイヤー
class Player:
  def __init__(self, name, coin):
    self.name = name
    self.coin = coin
    self.bets = {}
    self.reset_table()

  # ベットコイン設定
  def set_bet_coin(self, bet_coin, bet_cell):
    self.coin -= bet_coin
    self.bets[bet_cell] = bet_coin
    print('{}は{}コインを{}にBETしました。'.format(self.name, str(bet_coin), bet_cell))

  # テーブルリセット
  def reset_table(self):
    for cell in table:
      self.bets.update({cell.name: 0})

# 人間
class Human(Player):
  def __init__(self, name, coin):
    super().__init__(name, coin)

  # ベット
  def bet(self):
    if self.coin >= 99:
      max_bet_coin = 99
    else:
      max_bet_coin = self.coin
    bet_message = '何枚BETしますか？：(1-' + str(max_bet_coin) + ')'
    bet_coin = input(bet_message)
    while not self.enable_bet_coin(bet_coin, max_bet_coin):
      bet_coin = input(bet_message)

    bet_message = 'どこにBETしますか？：(R,B,1-8)'
    bet_cell = input(bet_message)
    while not self.enable_bet_cell(bet_cell):
      bet_cell = input(bet_message)
 
    super().set_bet_coin(int(bet_coin), bet_cell)

  # ベットコイン入力チェック
  def enable_bet_coin(self, string, max_bet_coin):
    if string.isdigit():
      number = int(string)
      if number >= 1 and number <= max_bet_coin:
        return True
      else:
        return False
    else:
      return False

  # ベット入力チェック
  def enable_bet_cell(self, string):
    if string.isdigit():
      number = int(string)
      if number >= 1 and number <= 8:
        return True
      else:
        return False
    else:
      if string == 'R' or string == 'B':
        return True
      else:
        return False

# コンピュータ
class Computer(Player):
  def __init__(self, name, coin):
    super().__init__(name, coin)

  # ベット
  def bet(self):
    if self.coin >= 99:
      max_bet_coin = 99
    else:
      max_bet_coin = self.coin
    bet_coin = random.randint(1, max_bet_coin)

    bet_cell_number = random.randint(0, len(cells) - 1)
    bet_cell = cells[bet_cell_number]
    super().set_bet_coin(bet_coin, bet_cell)

# セル
class Cell:
  def __init__(self, name, rate, color):
    self.name = name
    self.rate = rate
    self.color = color

# コマンドラインの色定義
class ColorBase:
  BLACK = '\033[30m'
  RED = '\033[31m'
  GREEN = '\033[32m'
  END = '\033[0m'

def set_cells():
  global cells
  cells = []
  for cell in table:
    cells.append(cell.__dict__['name'])

# プレイヤー作成
def create_players():
  global players
  human = Human('MY', 500)
  computer1 = Computer('C1', 500)
  computer2 = Computer('C2', 500)
  computer3 = Computer('C3', 500)
  players = [human, computer1, computer2, computer3]

def bet_players():
  for player in players:
    player.bet()

# あたりチェック
def check_hit():
  hit_cell_number = random.randint(0, len(cells) - 1)
  hit_cell = cells[hit_cell_number]
  print('選ばれたのは「{}」'.format(hit_cell))
  for player in players:
    if player.bets[hit_cell] >= 1:
      win_player(player, hit_cell_number)

# あたりプレイヤー判定
def win_player(player, hit_cell_number):
  hit_cell = cells[hit_cell_number]
  win_coin = player.bets[hit_cell] * table[hit_cell_number].rate
  player.coin += win_coin
  print(player.name + 'は当たり ' + str(win_coin) + 'コインを獲得しました。')

def show_coin():
  message = '[持ちコイン] '
  for player in players:
    message += player.name + ':' + str(player.coin) + ' / '
  print(message)

# テーブル作成
def create_table():
  global table
  table.append(Cell('R', 8, 'red'))
  table.append(Cell('B', 8, 'black'))
  table.append(Cell('1', 2, 'red'))
  table.append(Cell('2', 2, 'black'))
  table.append(Cell('3', 2, 'red'))
  table.append(Cell('4', 2, 'black'))
  table.append(Cell('5', 2, 'red'))
  table.append(Cell('6', 2, 'black'))
  table.append(Cell('7', 2, 'red'))
  table.append(Cell('8', 2, 'black'))

def show_table():
  row = green_bar() + '____' + green_bar()
  for player in players:
    row += player.name + green_bar()
  print(row)
 
  for cell in table:
    row = green_bar() + color(cell.color, cell.name + '(x' + str(cell.rate) + ')') + green_bar()
    for player in players:
      row += str(player.bets[cell.name]).zfill(2) + green_bar()
    print(row)

def reset_table():
  for player in players:
    player.reset_table()

def color(color_name, string):
  if color_name == 'red':
    return ColorBase.RED + string + ColorBase.END
  elif color_name == 'green':
    return ColorBase.GREEN + string + ColorBase.END
  else:
    return string

def green_bar():
  return color('green', '｜')

def initialize():
  create_table()
  create_players()
  set_cells()

def play_once():
  reset_table()
  bet_players()
  show_table()
  check_hit()
  show_coin()

def is_game_end():
  for player in players:
    if player.coin <= 0:
      return True
  return False

def game_end():
  for player in players:
    if player.coin <= 0:
      print(player.name + 'のコインがなくなったためゲームを終了します。')

def play():
  initialize()
  show_coin()
  while not is_game_end():
    play_once()
  else:
    game_end()

play()

[持ちコイン] MY:500 / C1:500 / C2:500 / C3:500 / 
何枚BETしますか？：(1-99)99
どこにBETしますか？：(R,B,1-8)1
MYは99コインを1にBETしました。
C1は61コインを1にBETしました。
C2は57コインを2にBETしました。
C3は58コインを3にBETしました。
｜____｜MY｜C1｜C2｜C3｜
｜R(x8)｜00｜00｜00｜00｜
｜B(x8)｜00｜00｜00｜00｜
｜1(x2)｜99｜61｜00｜00｜
｜2(x2)｜00｜00｜57｜00｜
｜3(x2)｜00｜00｜00｜58｜
｜4(x2)｜00｜00｜00｜00｜
｜5(x2)｜00｜00｜00｜00｜
｜6(x2)｜00｜00｜00｜00｜
｜7(x2)｜00｜00｜00｜00｜
｜8(x2)｜00｜00｜00｜00｜
選ばれたのは「1」
MYは当たり 198コインを獲得しました。
C1は当たり 122コインを獲得しました。
[持ちコイン] MY:599 / C1:561 / C2:443 / C3:442 / 
何枚BETしますか？：(1-99)99
どこにBETしますか？：(R,B,1-8)1
MYは99コインを1にBETしました。
C1は20コインを8にBETしました。
C2は69コインを8にBETしました。
C3は90コインを5にBETしました。
｜____｜MY｜C1｜C2｜C3｜
｜R(x8)｜00｜00｜00｜00｜
｜B(x8)｜00｜00｜00｜00｜
｜1(x2)｜99｜00｜00｜00｜
｜2(x2)｜00｜00｜00｜00｜
｜3(x2)｜00｜00｜00｜00｜
｜4(x2)｜00｜00｜00｜00｜
｜5(x2)｜00｜00｜00｜90｜
｜6(x2)｜00｜00｜00｜00｜
｜7(x2)｜00｜00｜00｜00｜
｜8(x2)｜00｜20｜69｜00｜
選ばれたのは「8」
C1は当たり 40コインを獲得しました。
C2は当たり 138コインを獲得しました。
[持ちコイン] MY:500 / C1:581 / C2:512 / C3:352 / 
何枚BETしますか？：(1-99)99
どこにBETしますか？：(R,B,1-8)1
MYは99コインを1にBETしました。
C1は39コインを1に